# FAKE NEWS
### Dataset Description

**_train.csv_**: A full training dataset with the following attributes:

- **id**: unique id for a news article
- **title**: the title of a news article
- **author**: author of the news article
- **text**: the text of the article; could be incomplete
- **label**: a label that marks the article as potentially unreliable
    - 1: unreliable
    - 0: reliable

**_test.csv_**: A testing training dataset with all the same attributes at train.csv without the label.

DATASET LINK: https://www.kaggle.com/c/fake-news/data

In [43]:
import nltk
import re
import pandas as pd 
import tensorflow as tf 
from nltk.corpus import stopwords
from tensorflow.keras.layers import Embedding 
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM 


In [2]:
df = pd.read_csv('train.csv')

In [3]:
df.head()

id                                              title              author  \
0   0  House Dem Aide: We Didn’t Even See Comey’s Let...       Darrell Lucus   
1   1  FLYNN: Hillary Clinton, Big Woman on Campus - ...     Daniel J. Flynn   
2   2                  Why the Truth Might Get You Fired  Consortiumnews.com   
3   3  15 Civilians Killed In Single US Airstrike Hav...     Jessica Purkiss   
4   4  Iranian woman jailed for fictional unpublished...      Howard Portnoy   

                                                text  label  
0  House Dem Aide: We Didn’t Even See Comey’s Let...      1  
1  Ever get the feeling your life circles the rou...      0  
2  Why the Truth Might Get You Fired October 29, ...      1  
3  Videos 15 Civilians Killed In Single US Airstr...      1  
4  Print \nAn Iranian woman has been sentenced to...      1

In [4]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [5]:
df = df.dropna()

In [6]:
X = df.drop('label', axis=1)
y = df['label']

In [7]:
vocabulary_size = 10000

In [8]:
# one-hot implementation
messages = X.copy()
messages.reset_index(inplace=True)


In [9]:
# Dataset preperation
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/anson/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [10]:
from nltk.stem.porter import PorterStemmer
ps_ = PorterStemmer()
corpus =[]

for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps_.stem(word) for word in review if not word in stopwords.words('english') ]
    review = ' '.join(review)
    corpus.append(review)


In [11]:
print(corpus, '\n')

['hous dem aid even see comey letter jason chaffetz tweet', 'flynn hillari clinton big woman campu breitbart', 'truth might get fire', 'civilian kill singl us airstrik identifi', 'iranian woman jail fiction unpublish stori woman stone death adulteri', 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart', 'beno hamon win french socialist parti presidenti nomin new york time', 'back channel plan ukrain russia courtesi trump associ new york time', 'obama organ action partner soro link indivis disrupt trump agenda', 'bbc comedi sketch real housew isi caus outrag', 'russian research discov secret nazi militari base treasur hunter arctic photo', 'us offici see link trump russia', 'ye paid govern troll social media blog forum websit', 'major leagu soccer argentin find home success new york time', 'well fargo chief abruptli step new york time', 'anonym donor pay million releas everyon arrest dakota access pipelin', 'fbi close hillari', 'chuck todd

In [12]:
# Embedding 
one_hot_representation = [one_hot(word,vocabulary_size)for word in corpus ]

In [13]:
one_hot_representation

[[7650, 3012, 9996, 1699, 57, 3492, 6144, 4533, 1294, 614],
 [2688, 7881, 1810, 1365, 1279, 7320, 3710],
 [6926, 715, 9088, 6250],
 [9997, 5630, 9922, 963, 5946, 2535],
 [7383, 1279, 8307, 2902, 1206, 2039, 1279, 2435, 3760, 528],
 [969,
  2687,
  5522,
  99,
  9474,
  2635,
  3415,
  8739,
  7878,
  1214,
  7575,
  1728,
  9632,
  4046,
  3710],
 [3611, 9476, 9071, 6938, 4291, 2196, 61, 7458, 1832, 7685, 5974],
 [9142, 900, 7722, 4166, 547, 1405, 2635, 7228, 1832, 7685, 5974],
 [9674, 8656, 548, 688, 613, 263, 9284, 6297, 2635, 7618],
 [3956, 2215, 2580, 2053, 1684, 3267, 3739, 2142],
 [9296, 9354, 4583, 2748, 1166, 1482, 638, 6902, 6980, 7518, 3912],
 [963, 2939, 57, 263, 2635, 547],
 [6480, 9258, 7515, 1146, 3537, 6893, 742, 8366, 6881],
 [7307, 1609, 8706, 2376, 7293, 1639, 9945, 1832, 7685, 5974],
 [7558, 4386, 9088, 487, 9398, 1832, 7685, 5974],
 [6894, 2946, 3286, 3811, 2049, 9196, 2849, 768, 1435, 7169],
 [2407, 9418, 7881],
 [1500, 356, 4369, 6274, 2635, 9551, 1440, 3710],
 [4

In [14]:
sent_length = 20
embedded_doc = pad_sequences(one_hot_representation, padding='pre', maxlen=sent_length)

In [15]:
embedded_doc

array([[   0,    0,    0, ..., 4533, 1294,  614],
       [   0,    0,    0, ..., 1279, 7320, 3710],
       [   0,    0,    0, ...,  715, 9088, 6250],
       ...,
       [   0,    0,    0, ..., 1832, 7685, 5974],
       [   0,    0,    0, ...,  875, 2859, 4281],
       [   0,    0,    0, ..., 9765, 4035, 6137]], dtype=int32)

In [27]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(vocabulary_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

/home/anson/miniconda3/envs/tf/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [28]:
len(embedded_doc)

18285

In [ ]:
# MODEL TRAINING

import numpy as np
X_final=np.array(embedded_doc)
y_final=np.array(y)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33)

In [30]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)


Epoch 1/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 6s 17ms/step - accuracy: 0.7788 - loss: 0.4382 - val_accuracy: 0.9203 - val_loss: 0.1812
Epoch 2/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - accuracy: 0.9446 - loss: 0.1361 - val_accuracy: 0.9274 - val_loss: 0.1747
Epoch 3/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.9681 - loss: 0.0933 - val_accuracy: 0.9226 - val_loss: 0.2050
Epoch 4/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.9776 - loss: 0.0656 - val_accuracy: 0.9176 - val_loss: 0.2099
Epoch 5/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.9852 - loss: 0.0481 - val_accuracy: 0.9210 - val_loss: 0.2401
Epoch 6/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.9897 - loss: 0.0322 - val_accuracy: 0.9226 - val_loss: 0.2744
Epoch 7/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - accuracy: 0.9942 - loss: 0.0187 - val_accuracy: 0.9158 - val_loss: 0.2966
Epoch 8/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.9954 - loss: 0.0163 - val_accu

In [41]:
y_pred = (model.predict(X_test) > 0.5).astype("int32")

189/189 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step


In [42]:

from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

array([[3125,  271],
       [ 244, 2395]])